# **1-1. 통합 테스트**

## **1.환경설정**

### (1) 경로 설정

로컬 경로 설정

In [ ]:
path = './'

### (2) 라이브러리

In [ ]:
#필요한 라이브러리 설치 및 불러우기
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import openai
from openai import OpenAI
import json
import torch
import sys
sys.path.append(path)

from transformers import AutoTokenizer, AutoModelForSequenceClassification


# 더 필요한 라이브러리 추가 -------------
import emergency as em

## **2. 단위 테스트**

* 세부사항 : 아래 단계별로 데이터가 순차적으로 처리되도록 단위 테스트를 진행합니다.

### (1) open ai key 등록

In [ ]:
openai.api_key = em.load_file(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key
print(os.environ['OPENAI_API_KEY'][:30])

### (2) audio to text

In [ ]:
audio_path = path + 'audio/'
filename = 'audio1.mp3'
result = em.audio_to_text(audio_path, filename)
result

### (3) text summary

In [ ]:
result = em.text_summary(result)
result

### (4) 응급실 등급분류

In [ ]:
# 모델, 토크나이저 로드
save_directory = path + "fine_tuned_bert"
model = AutoModelForSequenceClassification.from_pretrained(save_directory)
tokenizer = AutoTokenizer.from_pretrained(save_directory)

In [ ]:
predicted_class, probabilities = em.predict(result, model, tokenizer)

print(f"예측된 클래스: {predicted_class}")
print(f"예측된 클래스 이름: {predicted_class+1}등급")
print(f"클래스별 확률: {probabilities}")

### (5) 응급실추천

In [ ]:
emergency = pd.read_csv(path + '응급실 정보.csv')

map_key = em.load_file(path + 'map_key.txt')
map_key = json.loads(map_key)
c_id, c_key = map_key['c_id'], map_key['c_key']

In [ ]:
predicted_class = 2
start_lat, start_lng = 35.548238, 129.307011

if predicted_class <= 2 :
    result = em.recommend_hospital3(path, emergency, start_lat, start_lng, c_id, c_key)

result

## **3. 통합 테스트**

* 세부사항
    * [2. 단위 테스트] 의 내용을 순차적으로 실행하며 결과를 확인합니다.

In [ ]:
# 1. 준비 작업 ---------------------------------------------
openai.api_key = em.load_file(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key

map_key = em.load_file(path + 'map_key.txt')
map_key = json.loads(map_key)
c_id, c_key = map_key['c_id'], map_key['c_key']

emergency = pd.read_csv(path + '응급실 정보.csv')

# 모델, 토크나이저 로드
save_directory = path + "fine_tuned_bert"
model = AutoModelForSequenceClassification.from_pretrained(save_directory)
tokenizer = AutoTokenizer.from_pretrained(save_directory)


# 2. 데이터 처리(pipeline) ---------------------------------
# input
audio_path = path + 'audio/'
filename = 'audio1.mp3'

# 처리
result = em.audio_to_text(audio_path, filename)
result = em.text_summary(result)
predicted_class, _ = em.predict(result, model, tokenizer)

# 테스트 input --------------------
predicted_class = 2  # 테스트용
start_lat, start_lng = 35.548238, 129.307011
# ---------------------------------

if predicted_class <= 2 :
    result = em.recommend_hospital3(path, emergency, start_lat, start_lng, c_id, c_key)
    display(result)
else :
    print('개인 건강관리')